# code to import predictions for Types having 7 days predictions:



1 Wind
2 Solar
17 WindOffshore
20 Nuclear
21 WastePower
26 OtherPower



Import required dependencies

In [14]:
import os
import json
import requests
import pandas as pd
import pickle           # Save and load data
import time


# Setting Pandas options.
pd.set_option("display.max_rows", 50) # How to display all rows from data frame using pandas. Setting value to None to show all rows.
pd.set_option("display.max_columns", None)
pd.set_option("display.max_info_columns", 100)
pd.set_option("display.max_info_rows", 1000000)
pd.set_option("display.precision", 2)
#pd.set_option("styler.format.precision", 2)

load config files

In [15]:
# Load API Key and download directory from config file
CONFIG_FILE = "../config/api-call.json"

def load_config(config_path):
    """Load configuration from a JSON file."""
    with open(config_path, "r") as file:
        return json.load(file)

Config parameters for API calls

In [16]:
# Load config values
config = load_config(CONFIG_FILE)
# print(config)
API_ENDPOINT= config["ned"]["ned_api_endpoint"]
API_KEY = config["ned"]["demo-ned-api-key"]
DOWNLOAD_DIR = config["ned"]["ned_download_dir"]

# https://api.ned.nl/v1/utilizations?point=0&type=2&granularity=3&granularitytimezone=1&classification=2&activity=1&validfrom[strictly_before]=2020-11-17&validfrom[after]=2020-11-16


# Ensure the download directory exists
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# API details (Replace with actual API endpoint)
API_URL = API_ENDPOINT
#HEADERS = {"Authorization": f"X-AUTH-TOKEN {API_KEY}"}

# Headers with X-AUTH-TOKEN
api_headers = {
    "X-AUTH-TOKEN": API_KEY,  # Replace with your actual token
    "Content-Type": "application/json"  # Optional, if sending JSON
}

Make API calls in loop

In [25]:
from datetime import datetime, timedelta

current_datetime = datetime.now()
future_datetime = current_datetime + timedelta(days=8)



# Replace the datetime values with formatted strings only showing YYYY-MM-DD
current_datetime = current_datetime.strftime("%Y-%m-%d")
future_datetime = future_datetime.strftime("%Y-%m-%d")

print("Current datetime:", current_datetime)
print("Datetime 1 week from now:", future_datetime)

Current datetime: 2025-03-31
Datetime 1 week from now: 2025-04-08


In [ ]:
n_values = [1,2,17,20,21,26]

# Loop through the n values
for gen_type in n_values:
    api_params = {
        "point": "0",
        "type": gen_type,
        "granularity": "5",  # hourly
        "granularitytimezone": "1",
        "classification": "1",  # Forecast
        "activity": "1",
        "validfrom[before]": future_datetime,
        "validfrom[after]": current_datetime,
        "page": 1
    }

    # Initialize storage for full dataset
    full_data = []

    # First API request to determine the last page
    response = requests.get(API_URL, params=api_params, headers=api_headers)
    if response.status_code == 200:
        data = response.json()
        last_page_url = data.get("hydra:view", {}).get("hydra:last", None)
        
        if last_page_url:
            last_page_num = int(last_page_url.split("page=")[-1])
        else:
            print("Warning: Could not determine last page. Defaulting to 1.")
            last_page_num = 1  # Fallback to 1 if last_page_url is missing
        '''if last_page_url:
            last_page_num = int(last_page_url.split("page=")[-1])
            print("last_page_num:", last_page_num)
        else:
            print("Error: Could not determine last page.")
            exit()'''
    else:
        print(f"Error {response.status_code}: {response.text}")
        continue

    # Iterate through pages
    for page in range(1, last_page_num + 1):
        api_params["page"] = page
        response = requests.get(API_URL, params=api_params, headers=api_headers)
        
        if response.status_code == 200:
            json_dict = response.json()
            if "hydra:member" in json_dict:
                full_data.extend(json_dict["hydra:member"])
        else:
            print(f"Error on page {page}: {response.status_code}")
            break

    # After collecting all pages, convert the data to a DataFrame and add current_datetime column
    df = pd.DataFrame(full_data)
    df['current_datetime'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    file_path = os.path.join(DOWNLOAD_DIR, f"power-gen-pred-{gen_type}-{datetime.now().strftime('%Y%m%d_%H%M')}.csv")
    df.to_csv(file_path, index=False)

    print(f"Data collection complete! Saved {len(df)} records of Type-{gen_type} to csv")


Data collection complete! Saved 183 records to power-gen-pred-1.csv
Data collection complete! Saved 183 records to power-gen-pred-2.csv
Data collection complete! Saved 183 records to power-gen-pred-17.csv
Data collection complete! Saved 183 records to power-gen-pred-20.csv
Data collection complete! Saved 183 records to power-gen-pred-21.csv
Data collection complete! Saved 183 records to power-gen-pred-26.csv


# to add function to regularly make API calls, also need to add a timestamp